# A Map of crime in Skipton over 6 months
I recently moved into Skipton and have been deciding on where would be a good place to live and buy a house for me and my family. I decided to have a look at crime occurances over the last six months to see if crime varies noticably between different areas of Skipton.

## Data Sources
I took the crime data from the https://data.police.uk API, asking it to show me all the crimes that occured in a polygon roughly the shape of Skipton. I used Plotly Express to plot the crimes onto a map of Skipton from Mapbox. 

In [9]:
#import relevant libraries
import plotly.figure_factory as ff
import pandas as pd
#hide code that isnt aesthetic
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [10]:
#call the API to get the data for the area defined by the co-ords. One call for each month
calls = ['https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-09',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-08',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-07',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-06',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-05',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-04']

# Create an initial empty data frame
df = pd.DataFrame()
# Make the consecutive calls
for i, call in enumerate(calls):
    # Create the new DataFrame from the data you got
    df_new = pd.read_json(call, orient='records')
    # Merge it with the current data frame
    df = pd.concat([df, df_new])
#decrompess the nested data and add it to the table
df["latitude"] = df.latitude = pd.DataFrame(df.location.values.tolist())['latitude'].astype(float)
df["longitude"] = df.longitude = pd.DataFrame(df.location.values.tolist())['longitude'].astype(float)
#df['month'] =  pd.to_datetime(df['month'], format='%Y-%m')
#df = df.groupby(['month','id','category'])['location'].apply(','.join).reset_index()    

#make it look pretty with less superfluous crap
craven = df[['id', 'category', 'month', 'longitude', 'latitude']]

## Glance at the data
As can be seen below, in Skipton the majority of the crimes are anti social behaviour and violent crime with them accounting for 60.36 percent of all crime in Skipton during the 6 month time period.

In [11]:
#check out the data
temp = craven.groupby(['category'])['id'].count()
temp.to_frame().rename(columns={"id": "Crime Count"})

,Crime Count
category,
anti-social-behaviour,158
bicycle-theft,1
burglary,24
criminal-damage-arson,39
drugs,12
other-crime,4
other-theft,31
possession-of-weapons,1
public-order,26


In [12]:
#find out the percentage of crimes that are violent crime and anti social behaviour
sum_vio_ant = temp.transpose()['anti-social-behaviour'] + temp.transpose()['violent-crime']
sum_all = temp.sum(axis=0)
percent = (sum_vio_ant/sum_all)*100
print(percent)

60.36363636363637


## Plot data to a map

I plotted the crime data to a map, with a slider allowing the different crime types to be displayed and the colour of the dots displaying the month.

### Current issues:

The labels on the right are ugly
The slider bar is ugly
The labels seem to show multiple of a single month
The slider seems to display multiple types of crime even when set to a single crime
The points on the map are difficult to see compared to the back colour



In [13]:
import plotly.express as px
fig = px.scatter_mapbox(craven, lon="longitude", lat="latitude", color="month", zoom=14, animation_frame="category")
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [17]:
import numpy as np

violent = craven.loc[craven['category']=="violent-crime"]
#need to figure out how to aggregate the group crime types and count whats in each then add it to the dataframe
#reset index is there in order to get the name for the count column
#longitutde and latitude need to be inside the square brackets or it messes up.
grouped = violent.groupby(['longitude','latitude','month']).size().reset_index(name='count')
#grouped['freq'] = violent.groupby('longitude')['latitude'].value_counts().transform('freq')

# Violent Crimes in Skipton by month

I created a different figure, this time with the size of the point showing the count of the crime and the colour showing the month comitted. This is only for violent crimes, but it gives a clear picture of the locations. Still has some of the flaws of the above figure.


In [15]:
#Created a different figure, this time with the size of the point showing the count of the crime and the colour showing the month comitted
fig2 = px.scatter_mapbox(grouped, lon="longitude", lat="latitude", color="month", 
                         size="count", zoom=14)
fig2.update_layout(mapbox_style="open-street-map")
fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig2.update_layout(
    title="Violent Crimes in Skipton by month",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig2.show()

To do list:

1) Put the data into github and read the CSV from there so that sam can access it
2) make sure the data is fully readable, removing the spaces commas in the thousands. The thousand commas can be removed during the read CSV and the spaces can be removed with a replace function.
3) use plotly to make the map

call the data from an API
Probably can call the location id's from an api as well

In [16]:
fig3 = px.scatter_mapbox(grouped, lon="longitude", lat="latitude", 
                         size="count", zoom=14, animation_frame="month")
fig3.update_layout(mapbox_style="open-street-map")
fig3.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig3.update_layout(
    title="Violent Crimes in Skipton by month",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"    )
)

fig3.show()

Here is a pointer on his to fix the date order. Specify the frame sequence.

Defining Button Arguments
None: Setting "args" to undefined (i.e. "args": [None]) will create a simple play button that will animate all frames.
string: Animate all frames with group "<some string>". This is a way of scoping the animations in case you would prefer to animate without explicitly enumerating all frames.
["frame1", "frame2", ...]: Animate a sequence of named frames.